In [4]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed) 
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False
torch.set_num_threads(6)


def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            'model': 'MLP',
            'random_class_idx' : False,

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 5,
            'per_task_examples': np.inf,
            # 'per_task_examples': 10000,
            'per_task_memory_examples': 64,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha': 0.00,
            'metric' : "EO",
            'lambda': 1.0,
            'lambda_old': 0.0,
              }
    


#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_tau={params['tau']}_alpha={params['alpha']}"
    if params['lambda'] != 0:
        trial_id+=f"_lmbd_{params['lambda']}_lmbdold_{params['lambda_old']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=MNIST/seed=10_epoch=5_lr=0.001_tau=10.0_alpha=0.0_lmbd_1.0_lmbdold_0.0


In [5]:
"MNIST" in params['dataset']

True

In [6]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                    per_task_memory_examples=params['per_task_memory_examples'],
                    per_task_examples = params['per_task_examples'],
                    random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)



[0 1 2 3 4 5 6 7 8 9]


In [7]:
from trainers import FairContinualTrainer
from trainers.fair_trainer import FairContinualTrainer1
# from trainers.baselines import BaseMemoryContinualTrainer as ContinualTrainer
from metrics import FairMetricCollector
from algorithms.fairl import FaIRL
from algorithms.fairl import FaIRL
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])
algorithm = FaIRL(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

trainer = FairContinualTrainer1(algorithm, params, callbacks=[metric_manager_callback])


FaIRL


In [8]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.9937157241751327, 'loss': 0.0032638977887219007, 'std': 0.0018789894812550334, 'EER': -1, 'EO': -1, 'DP': -1, 'accuracy_s0': -1, 'accuracy_s1': -1, 'classwise_accuracy': {0: array([972, 980]), 1: array([1130, 1135])}, 'DP_ingredients': {}}
[2] Eval metrics for task 1 >> {'accuracy': 0.9921851119302346, 'loss': 0.0020663118954245925, 'std': 0.003409601726153011, 'EER': -1, 'EO': -1, 'DP': -1, 'accuracy_s0': -1, 'accuracy_s1': -1, 'classwise_accuracy': {1: array([1130, 1135]), 0: array([969, 980])}, 'DP_ingredients': {}}
[3] Eval metrics for task 1 >> {'accuracy': 0.992254787377506, 'loss': 0.0015517447857146568, 'std': 0.002458869010159126, 'EER': -1, 'EO': -1, 'DP': -1, 'accuracy_s0': -1, 'accuracy_s1': -1, 'classwise_accuracy': {0: array([970, 980]), 1: array([1129, 1135])}, 'DP_ingredients': {}}
[4] Eval metrics for task 1 >> {'accuracy': 0.9942259282567653, 'loss': 0.00130318524

In [ ]:
metric_manager_callback.meters['accuracy'].get_data()

array([[89.873,  0.   ,  0.   ,  0.   ,  0.   ],
       [83.669, 49.881,  0.   ,  0.   ,  0.   ],
       [56.745, 59.669, 49.534,  0.   ,  0.   ],
       [51.395, 49.234, 48.427, 12.365,  0.   ],
       [46.27 , 40.235, 40.714, 24.429, 44.758]])

In [ ]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

58.31999894627133

In [ ]:
[np.round(x, 3) for x in metric_manager_callback.meters['EO'].compute_overall()]

39.28119091305616

In [ ]:
np.mean(metric_manager_callback.meters['EO'].compute_overall())

[0.252, 0.356, 0.572, 0.714, 0.689]

In [ ]:
[np.round(x, 3) for x in metric_manager_callback.meters['DP'].compute_overall()]

0.5165693908539541

In [ ]:
np.mean(metric_manager_callback.meters['DP'].compute_overall())

task_id=2
sensitive samples / all samples = 644 / 12700
sensitive samples / selected samples = 503 / 10000
